In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from datastructuretools.cache import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [3]:
from dataviztools import bokehutils
from bokeh.plotting import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [4]:
import importlib
importlib.reload(bokehutils)

<module 'dataviztools.bokehutils' from '/hosthome/Workspace/Python/Utils/DatavizTools/dataviztools/bokehutils.py'>

In [9]:
from machinelearning.function import *

In [5]:
logger = Logger()

# Printing stats on rankings

In [11]:
twinewsRankings = getTwinewsRankings()
models = set()
for row in twinewsRankings.collection.find({"meta.splitVersion": 2, 'meta.maxUsers': None}):
    if row['meta']['model'] != 'combin':
        models.add(row['meta']['model'])
models.remove('worst')
models.remove('ideal')
models.remove('random')
keys = set()
for model in models:
    keys.add(twinewsRankings.collection.find_one({'meta.model': model, "meta.splitVersion": 2, 'meta.maxUsers': None})['id'])
# keys.add("combin-8f846")
# keys.add("combin-efad4")
keys = {'dbert-base-fe9f3', 'sent2vec-7e0f9'}
keys = {'dbert-base-fe9f3'}
keys = sorted(list(keys))
print(keys)

['dbert-base-fe9f3']


In [13]:
def rankingReport(rankings, maxUsers=None, logger=None, verbose=True, pbarVerbose=True):
    if maxUsers is not None:
        keys = shuffle(list(rankings.keys()), seed=0)[:maxUsers]
        rankings = dictSelect(rankings, keys)
    # We find all data:
    medians = []
    means = []
    rkvs = []
    scoress = []
    allScores = []
    for userId in pb(list(rankings.keys()), logger=logger, verbose=pbarVerbose):
        assert isinstance(rankings[userId][0][0], tuple)
        scores = [e[1] for e in rankings[userId][0]]
        scores = normalizeRankingScores(scores)
        scores = normalizedLawX(scores, alpha=0.2, beta=NormalizedLawBeta.EXP, inverse=False)
        medians.append(np.median(scores))
        means.append(np.mean(scores))
        rkvs.append(rankingVariance(scores))
        scoress.append(scores)
        allScores += list(scores)
    # We print means and std:
    for values, label in \
    [
        (medians, "Median"), (means, "Mean"),
        (rkvs, "Ranking variance")
    ]:
        std = np.std(values)
        # std = mean_confidence_interval(values)
        std = truncateFloat(std, 2)
        mean = truncateFloat(np.mean(values), 2)
        text = label + ": " + str(mean) + " ±" + str(std) + " (95%)"
        log(text, logger, verbose=verbose)
    # We show all scores histogram:
    log("Histogram of all scores (as distances):", logger=logger, verbose=verbose)
    # show(bokehutils.barplot(allScores, 30, hideTools=True, hideXAxis=True, hideYAxis=True, title="", height=100, width=300))
    show(bokehutils.barplot(allScores, 30, hideTools=False, hideXAxis=True, hideYAxis=True, title="", height=100*2, width=300*2))
    # We show a sample of scores:
    log("Histogram samples of scores (as distances):", logger=logger, verbose=verbose)
    scoress = shuffle(list(scoress))
    figures = []
    for row in range(3):
        currentFigures = []
        for col in range(3):
            scores = scoress.pop()
            f = bokehutils.barplot(scores, 10, hideTools=True, hideXAxis=True, hideYAxis=True, title="", height=40, width=100)
            currentFigures.append(f)
        figures.append(currentFigures)
    grid = gridplot(figures, toolbar_options={'logo': None})
    show(grid)

In [14]:
tt = TicToc()
tt.tic()
for key in keys:
    print("#" * 20 + " " + key + " " + "#" * 20)
    rankings = twinewsRankings[key]
    tt.tic("Got " + key + " rankings")
    if rankings is None:
        print("No rankings")
    else:
        if isinstance(rankings[list(rankings.keys())[0]][0][0], tuple):
            try:
                rankingReport(rankings, maxUsers=None, pbarVerbose=False)
            except Exception as e:
                logException(e)
            tt.tic("Stats printed for " + key)
        else:
            print("No scores")
tt.toc()

--> tictoc starts...
#################### dbert-base-fe9f3 ####################
--> tic: 18.08s | message: Got dbert-base-fe9f3 rankings
Median: 0.41 ±0.1 (95%)
Mean: 0.42 ±0.09 (95%)
Ranking variance: 0.0 ±0.0 (95%)
Histogram of all scores (as distances):


Histogram samples of scores (as distances):


--> tic: 1m 45.129s | message: Stats printed for dbert-base-fe9f3
--> toc total duration: 2m 3.209s


123.21

# Affichage des dominances

In [ ]:
keys

In [ ]:
def getCombination(m1, m2, splitVersion=1):
    c = twinewsGet("combin", splitVersion=splitVersion)
    results = []
    for current in c:
        if m1 in current['models'] and m2 in current['models'] and current['rankAsScore'] == [False, False]:
            results.append(current)
    assert len(results) == 1
    return results[0]

In [ ]:
def getDominance(*args, **kwargs):
    return getCombination(*args, **kwargs)['dominance']

In [ ]:
def printDominance(*args, **kwargs):
    d = getDominance(*args, **kwargs)
    print(str(args) + " --> " + str(truncateFloat(d, 2)))

In [ ]:
lefts = ['bert-a7c9a', 'dbert-base-d092a', 'infersent-77ec7', 'stylo-5c321']
rights = ['bm25-933f7', 'lda-82272', 'nmf-6078e', 'tfidf-4a1dc']
middle = ['dbert-ft-7847a', 'sent2vec-32f92', 'doc2vec-e013a', 'usent-64ec0']

In [ ]:
set1 = lefts
set2 = middle

In [ ]:
for m1 in set1 + set2:
    for m2 in set1 + set2:
        if m1 != m2:
            c = getCombination(m1, m2)
            combText = ""
            model1 = c['models'][0]
            model2 = c['models'][1]
            m1IsSet1 = model1 in set1
            m2IsSet1 = model2 in set1
            combText += model1
            combText += " (set1)" if m1IsSet1 else " (set2)"
            combText += " "
            combText += model2
            combText += " (set1)" if m2IsSet1 else " (set2)"
            d = getDominance(m1, m2)
            combText += " --> " + str(truncateFloat(d, 2))
            if m1IsSet1 and not m2IsSet1:
                print(combText)

In [ ]:
printDominance('bert-a7c9a', 'bm25-933f7')

# Affichage d'exemples de courbes de scores

In [ ]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
import numpy as np
output_notebook()

In [ ]:
twinewsRankings = getTwinewsRankings()

In [ ]:
rankings = twinewsRankings['lda-d4f1d'] # TODO prendre LDA de split version 2

In [ ]:
u1Rankings = rankings[random.choice(list(rankings.keys()))][0]

In [ ]:
values = [x[1] for x in u1Rankings]

In [ ]:
# Averaging:
values = [0] * 1000
for id, r in rankings.items():
    r = r[0]
    for i in range(1000):
        values[i] += r[i][1]

In [ ]:
bp(values)

In [ ]:
values = normalizeRankingScores(values)

In [ ]:
bp(list(values))

In [ ]:
green = "#009b95"
red = "#f25252"
blue = "#007acc"
purple = "#b400cc"

In [ ]:
n = 1000
w = 3
X = list(range(0, 1000))

In [ ]:
p = figure(plot_width=400, plot_height=400)
p.xaxis.axis_label = 'Item position'
p.yaxis.axis_label = 'Score'
p.line(X, values, line_width=w, color="grey", legend_label="Original LDA scores")
values2 = normalizedLawX(values, alpha=0.93, beta=NormalizedLawBeta.LOG, inverse=False)
p.line(X, values2, line_width=w, color=green, legend_label="α = 0.93, β = True")
values2 = normalizedLawX(values, alpha=0.94, beta=NormalizedLawBeta.EXP, inverse=False)
p.line(X, values2, line_width=w, color=blue, legend_label="α = 0.94, β = False")
p.legend.location = 'bottom_right'
show(p)

In [ ]:
import bokeh
print(bokeh.__version__)

# Avantage d'un modèle dans une combinaison

In [ ]:
def getDominance(combin, logger=None, verbose=False):
    meta = twinewsRankings.getMeta(combin)
    model1, model2 = meta['models'][0], meta['models'][1]
    combinRankings = rankingsCache[combin]
    model1Rankings = rankingsCache[model1]
    model2Rankings = rankingsCache[model2]
    userIds = list(combinRankings.keys())
    domin1 = 0
    domin2 = 0
    dominationRatio1 = None
    for i in range(1000000):
        userId = random.choice(userIds)
        crk = pruneScores(combinRankings[userId][0])
        m1rk = pruneScores(model1Rankings[userId][0])
        m2rk = pruneScores(model2Rankings[userId][0])
        urlIndex = getRandomInt(0, 999)
        url = crk[urlIndex]
        position1 = m1rk.index(url)
        position2 = m2rk.index(url)
        distance1 = abs(position1 - urlIndex)
        distance2 = abs(position2 - urlIndex)
        if distance1 == distance2:
            pass
        elif distance1 < distance2:
            domin1 += 1
        else:
            domin2 += 1
        if i > 1000 and i % 10000 == 0:
            newDominationRatio1 = domin1 / (domin1 + domin2)
            if dominationRatio1 is not None:
                if abs(dominationRatio1 - newDominationRatio1) < 0.0005:
                    break
            dominationRatio1 = newDominationRatio1
            log("dominationRatio1: " + str(truncateFloat(dominationRatio1, 2)), logger, verbose=verbose)
            log('-' * 10, logger, verbose=verbose)
    dominationRatio1 = domin1 / (domin1 + domin2)
    return dominationRatio1

In [ ]:
twinewsRankings = getTwinewsRankings()
rankingsCache = Cache(twinewsRankings.get, minFreeRAM=20, logger=logger)

In [ ]:
combinsSplit1 = getCombinIds(1)
combinsSplit2 = getCombinIds(2)
bp(combinsSplit1, logger)
bp(combinsSplit2, logger)

In [ ]:
dominancesSD = getDominancesSD()
bp(list(dominancesSD.items()), logger)

In [ ]:
for combin in pb(list(combinsSplit1) + list(combinsSplit2), logger=logger, printRatio=0.01):
    if combin not in dominancesSD:
        meta = twinewsRankings.getMeta(combin)
        model1, model2 = meta['models'][0], meta['models'][1]
        rankAsScore = meta['rankAsScore']
        log("#" * 10 + " " + combin + " (" + model1 + " / " + model2 + ") " + "#" * 10, logger)
        log("rankAsScore: " + str(rankAsScore), logger)
        dominationRatio1 = getDominance(combin, logger=logger, verbose=False)
        log("Dominance prob of " + model1 + ": " + str(truncateFloat(dominationRatio1, 2)), logger)
        # log("#" * 60, logger)
        dominancesSD[combin] = dominationRatio1

# Trash

In [ ]:
if False:
    def getDominances(combin):
        meta = twinewsRankings.getMeta(combin)
        model1, model2 = meta['models'][0], meta['models'][1]
        combinRankings = rankingsCache[combin]
        model1Rankings = rankingsCache[model1]
        model2Rankings = rankingsCache[model2]
        userIds = list(combinRankings.keys())
        p1s = []
        p2s = []
        p1 = None
        p2 = None
        for i in range(1000000):
            userId = random.choice(userIds)
            crk = pruneScores(combinRankings[userId][0])
            m1rk = pruneScores(model1Rankings[userId][0])
            m2rk = pruneScores(model2Rankings[userId][0])
            urlIndex = getRandomInt(0, 999)
            url = crk[urlIndex]
            position1 = m1rk.index(url)
            position2 = m2rk.index(url)
            p1s.append(position1 < urlIndex)
            p2s.append(position2 < urlIndex)
            if i % 10000 == 0:
                newp1 = p1s.count(True) / len(p1s)
                newp2 = p2s.count(True) / len(p2s)
                if p1 is not None and p2 is not None:
                    if abs(p1 - newp1) < 0.001 and abs(p2 - newp2) < 0.001:
                        break
                p1, p2 = newp1, newp2
                # print("p1: " + str(truncateFloat(p1, 2)))
                # print("p2: " + str(truncateFloat(p2, 2)))
                # print('-' * 10)
        p1 = p1s.count(True) / len(p1s)
        p2 = p2s.count(True) / len(p2s)
        return (p1, p2)